Numba's decorators
=============

Numba is a family of decorators and a couple of special types. We've seen `@njit` already, this is for JIT compiling a function and is in actual fact an alias for `@numba.jit(nopython=True)`.

## Some history...
A long while ago Numba had the concept of fall-back modes added to JIT compilation, it would do the following:
1. Try and compile the function in `nopython` mode, `nopython` mode is what it sounds like, a mode where there is no-python involved, it's all machine code.
2. If `nopython` mode failed, Numba would try again compiling in "loop-lifting" mode, this is where Numba will scan the code for loops and try and compile those.
3. If "loop-lifting" mode failed Numba would compile the code into C-API calls, which is not particularly useful and can even be slower.

Currently Numba is in the process of deprecating this as it's a headache for compiler engineers and seriously confusing for users as seemingly simple code changes can drastically alter performance.

Back to the decorators...

@vectorize
----------

NumPy is built on Universal Functions ("ufuncs") as a key concept, these functions perform their operations element wise on the input and handle all the broadcasting over arrays etc. What if you want to define your own ufunc? Meet Numba's `@vectorize`.

In [ ]:
import numpy as np
from numba import vectorize, njit

@vectorize
def ReynoldsNo(rho, u, L, mu):
    return (rho * u * L) / mu

rho = 1000 # density of pure water
speed = 2 # human swimming
length = 1.8 # human height
dyn_visc = 1e-3 # dynamic viscosity of water at about 20 C

ReynoldsNo(rho, speed, length, dyn_visc)

How fast do you need to swim to match the Reynold number of a blue whale?

In [ ]:
u = np.arange(0, 1000, 10)
re = ReynoldsNo(rho, u, length, dyn_visc)

blue_whale = 4e8
u[np.argmin(np.abs(re - blue_whale))]

By default all `@vectorize` ufuncs are "Dynamic ufuncs", which means they will recompile themselves on encountering a new type. Express "length" in the complex domain just as an example:

In [ ]:
ReynoldsNo(rho, speed, 2j, dyn_visc)

Check the type signatures cf. NumPy:

In [ ]:
ReynoldsNo.types

### Switching the target backend...

The vectorize decorator was just targetting a single core of a CPU, this is the default target. Other targets of interest include:
 * "cpu" - serial CPU execution
 * "parallel" - parallel CPU execution
 * "CUDA" - NVIDIA CUDA
 * "ROC" - AMD ROCm
 
Let's try out the CPU targets with the classic form of solving:

$ a x^2 + b x + c = 0 $

Using the formula for returning the positive solution:

$ x = \frac{-b + \sqrt{b^2 - 4ac}}{2a} $

In [ ]:
# some inputs
from math import sqrt
n = 10000
a = np.random.random((n,))
b = 4 + np.random.random((n,))
c = np.random.random((n,))

<h3><span style="color:blue"> Task 1a: Implement the above equation as a `@vectorize` function</span></h3>

In [ ]:
# <Add the decorator here>
def my_solver(a, b, c):
    # You write this!

# check it's correct
assert my_solver(3, 10, 3) == -3

<h3><span style="color:blue"> Task 1b: Measure the performance of your implementation</span></h3>

In [ ]:
%timeit my_solver(a, b, c)

Some targets require signatures still (this is a hang over from older code paths), a signature is a way to spell out the the compiler the permitted types of the inputs (and outputs). In the case above, there's three "float64" scalar inputs and a "float64" scalar output. If you ever aren't sure of the type of something, use `numba.typeof()` and it'll tell you. The types can be spelled as strings or as instances of `numba.types` classes. It's these classes that you'll see in the type inference system/error message feedback.

<h3><span style="color:blue"> Task 1c: Use your function body from above in the stub below</span></h3>

In [ ]:
sig = "float64(float64, float64, float64,)"
@vectorize([sig,], target="parallel")
def my_solver_parallel(a, b, c):
    # <copy your solver function body here>

<h3><span style="color:blue"> Task 1d: Check the performance of the function on the "parallel" target</span></h3>

In [ ]:
%timeit my_solver_parallel(a, b, c)

@guvectorize
----------

Numba extends the ufunc concept to "Generalised ufuncs" these are indeed a generalisation on "ufuncs" in that they allow arrays as inputs but still handle all the broadcasting etc. Things to note:
 * You have to spell out the type signature(s)
 * You have to supply an input/output layout (this is so the compiler knows how to generate the inner loop nest).
 * The function has a `void` return type, the layout specification is used to determine which arguments are inputs/outputs.
 
Using matrix multiplication solely as a pedagogical example (naturally, you'd use `np.dot`!):

In [ ]:
from numba import guvectorize

@guvectorize(["float64[:,:], float64[:,:], float64[:,:]",], '(m, k),(k, n)->(m, n)')
def matmul(x, y, res):
    m, k = x.shape
    k2, n = y.shape
    out_m, out_n = res.shape
    assert k == k2
    assert m == out_m
    assert n == out_n
    for p in range(m):
        for q in range(n):
            res[p, q] = 0
            for r in range(k):
                res[p, q] += x[p, r] * y[r, q]


m, n, k = 3, 4, 5
a = np.arange(m * k).reshape(m, k)
b = np.arange(k * n).reshape(k, n)
out = np.zeros((m, n))

expected = np.dot(a, b)

matmul(a, b, out)
np.testing.assert_allclose(out, expected)

It should be noted that the layout specification is the layout of the data needed for the kernel (the function body) to operate, **not** the layout of the arrays supplied as input. The `@guvectorize` decorator will do all the broadcasting necessary to use lower dimensioned kernels on higher dimensioned data. For example, using the `matmul` function above on some 3D input, note how the kernel is broadcast across the two inner most dimensions (as determined by the layout specification).

In [ ]:
outer, m, n, k = 7, 3, 4, 5
a = np.arange(outer * m * k).reshape(outer, m, k)
b = np.arange(outer * k * n).reshape(outer, k, n)
out = np.zeros((outer, m, n))
expected = np.zeros((outer, m, n))

# np.dot has different broadcasting rules to the kernel above
for x in range(outer):
    expected[x] = np.dot(a[x], b[x])

matmul(a, b, out)

np.testing.assert_allclose(out, expected)

The `@guvectorize` decorator supports the same targets as the `@vectorize` decorator.

@stencil
--------

Stencil operations are so common that Numba provides a special decorator to help `@stencil`. It by default uses absolute indexing and handles broadcasting and loop generation etc automatically. Example:`

In [ ]:
from numba import stencil

@stencil
def weighted_3_mean(a):
    return (a[-1] + 2 * a[0] + a[1])/3

z = np.arange(10)
weighted_3_mean(z)

Note that the start and end values are 0, the kernel does not extend to these. Limited boundary condition support is available along with a host of other features, see the [docs](http://numba.pydata.org/numba-doc/latest/user/stencil.html).

Automatic parallelism
-------------
We've seen that Numba has a parallel target accessible, this is also available for the `@jit` family and is switched on via the kwarg `parallel=True`. Revisit `awkward_sine` from the first notebook...

In [ ]:
n = 10000

def awkward_sine(n):
    a = np.ones(n)
    return np.imag(np.exp(1j * a))

Check the performance of the NumPy based function:

In [ ]:
%timeit awkward_sine(n)

JIT compile the function and check the performance of the JIT compiled version:

In [ ]:
njit_awkward_sine = njit()(awkward_sine)

%timeit njit_awkward_sine(n)

JIT compile the function with `parallel=True` set and check the performance of this:

In [ ]:
parallel_njit_awkward_sine = njit(parallel=True)(awkward_sine)

%timeit parallel_njit_awkward_sine(n)

Numba has some parallel diagnostic output available, it's accessible via the `.parallel_diagnostics()` method on the dispatcher [docs](http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics). 

In [ ]:
parallel_njit_awkward_sine.parallel_diagnostics(level=4)

The above is implicit parallelism, array analysis is being done to work out what can be executed safely in parallel. Explicit parallelism is also possible using a `numba.prange` driven loop. The `prange` function is a like `range` but will cause the loop body to execute in parallel if the `parallel=True` target is in use (else it just behaves like normal `range`).


<h3><span style="color:blue"> Task 2a: Compute the 2-norm of this vector using a `prange` loop with an accumulator</span></h3>

For reference the 2-norm of vector $x$ is:

$|x|_2 = \sqrt{\sum_{i=0}^{n} {x_i}^2}$

In [ ]:
from numba import prange

@njit(parallel=True)
def two_norm(x):
    # You write this!


x = np.arange(100.)

np.testing.assert_allclose(two_norm(x), np.linalg.norm(x))

<h3><span style="color:blue"> Task 2b: Investigate the performance</span></h3>

Some `timeit` helpers, first the python function, second the `np.linalg.norm` implementation from NumPy and last, the parallel implementation above.

In [ ]:
%timeit two_norm.py_func(x)

In [ ]:
%timeit np.linalg.norm(x)

In [ ]:
%timeit two_norm(x)